In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

ROOT = Path("..")
raw_parquet = ROOT / "data" / "raw" / "yellow_tripdata_2024-01.parquet"

df = pd.read_parquet(raw_parquet)
print(df.shape)
df.head()


(2964624, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [3]:
df2 = df.copy()

# Passenger count: 0 means unknown -> set to NaN
df2.loc[df2["passenger_count"] == 0, "passenger_count"] = np.nan

# Payment type: 0 means unknown -> keep, but tag later
# We'll handle it as a category, not a deletion.

In [9]:
duration_min = (df2["tpep_dropoff_datetime"] - df2["tpep_pickup_datetime"]).dt.total_seconds() / 60
pickup = df2["tpep_pickup_datetime"]
dropoff = df2["tpep_dropoff_datetime"]

bad_year = (pickup.dt.year < 2017) | (pickup.dt.year > 2025)
bad_month = ~pickup.dt.month.isin(range(1, 13))  # mostly redundant, but harmless

rules = {
    # Hard invalid (remove)
    "bad_datetime_order": duration_min <= 0,
    "bad_trip_distance": (df2["trip_distance"].isna()) | (df2["trip_distance"] <= 0),
    "negative_fare": df2["fare_amount"] < 0,
    "negative_total": df2["total_amount"] < 0,
    "negative_tip": df2["tip_amount"] < 0,

    # Soft quality flags (keep, but track)
    "too_long_duration": duration_min > 6 * 60,
    "extreme_distance": df2["trip_distance"] > 200,
    "unknown_payment_type": df2["payment_type"] == 0,
    "high_passenger_count": df2["passenger_count"].notna() & (df2["passenger_count"] > 8),
    "invalid_ratecode": df2["RatecodeID"].notna() & ~df2["RatecodeID"].isin([1,2,3,4,5,6]),
    "invalid_store_flag": df2["store_and_fwd_flag"].notna() & ~df2["store_and_fwd_flag"].isin(["Y","N"])
}

In [10]:
report = []
n = len(df)

for name, mask in rules.items():
    count = int(mask.sum())
    report.append({
        "rule": name,
        "rows_flagged": count,
        "percent_flagged": round((count / n) * 100, 3)
    })

dq_report = pd.DataFrame(report).sort_values("rows_flagged", ascending=False)
dq_report


,rule,rows_flagged,percent_flagged
7,unknown_payment_type,140162,4.728
1,bad_trip_distance,60371,2.036
2,negative_fare,37448,1.263
3,negative_total,35504,1.198
9,invalid_ratecode,28663,0.967
5,too_long_duration,1772,0.060
0,bad_datetime_order,870,0.029
4,negative_tip,102,0.003
6,extreme_distance,31,0.001
8,high_passenger_count,1,0.000


In [11]:
hard_fail = (
    rules["bad_datetime_order"] |
    rules["bad_trip_distance"] |
    rules["negative_fare"] |
    rules["negative_total"] |
    rules["negative_tip"]
)
hard_fail = hard_fail | bad_year

df_clean = df2.loc[~hard_fail].copy()

# Compute duration for df_clean (used for long-duration flag)
duration_min_clean = (
    (df_clean["tpep_dropoff_datetime"] - df_clean["tpep_pickup_datetime"])
    .dt.total_seconds() / 60
)

# Soft-quality flags (do NOT delete rows)
df_clean["flag_unknown_payment_type"] = (df_clean["payment_type"] == 0)
df_clean["flag_long_duration"] = (duration_min_clean > 360)          # > 6 hours
df_clean["flag_extreme_distance"] = (df_clean["trip_distance"] > 200)
df_clean["flag_high_passenger_count"] = (
    df_clean["passenger_count"].notna() & (df_clean["passenger_count"] > 8)
)

print("Before:", df2.shape)
print("After :", df_clean.shape)
print("Removed:", len(df2) - len(df_clean))
print("Percent removed:", round((len(df2) - len(df_clean)) / len(df2) * 100, 3), "%")

print("\nFlag counts (soft issues kept):")
print(df_clean[[
    "flag_unknown_payment_type",
    "flag_long_duration",
    "flag_extreme_distance",
    "flag_high_passenger_count"
]].sum())

df_clean["pickup_year"] = df_clean["tpep_pickup_datetime"].dt.year
df_clean["pickup_month"] = df_clean["tpep_pickup_datetime"].dt.month


Before: (2964624, 19)
After : (2869992, 23)
Removed: 94632
Percent removed: 3.192 %

Flag counts (soft issues kept):
flag_unknown_payment_type    115239
flag_long_duration             1744
flag_extreme_distance            31
flag_high_passenger_count         1
dtype: int64


we reduced almost 3% of rows with bad data breaking the hard rules of negative values


In [12]:
out_dir = ROOT / "data" / "processed" / "yellow_cleaned_parquet"

df_clean.to_parquet(
    out_dir,
    engine="pyarrow",
    index=False,
    partition_cols=["pickup_year", "pickup_month"]
)

print("Wrote cleaned parquet to:", out_dir)


Wrote cleaned parquet to: ..\data\processed\yellow_cleaned_parquet


In [13]:
df_back = pd.read_parquet(out_dir)
print(df_back.shape)
df_back

(11479980, 25)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,flag_unknown_payment_type,flag_long_duration,flag_extreme_distance,flag_high_passenger_count,pickup_year,pickup_month
0,2,2002-12-31 22:59:39,2002-12-31 23:05:41,1.0,0.63,1.0,N,170,170,3,...,1.0,10.50,2.5,0.00,False,False,False,False,2002,12
1,2,2002-12-31 22:59:39,2002-12-31 23:05:41,1.0,0.63,1.0,N,170,170,3,...,1.0,10.50,2.5,0.00,False,False,False,False,2002,12
2,2,2002-12-31 22:59:39,2002-12-31 23:05:41,1.0,0.63,1.0,N,170,170,3,...,1.0,10.50,2.5,0.00,False,False,False,False,2002,12
3,2,2009-01-01 23:58:40,2009-01-02 00:01:40,1.0,0.46,1.0,N,137,264,2,...,1.0,9.40,0.0,0.00,False,False,False,False,2009,1
4,2,2009-01-01 23:30:39,2009-01-02 00:01:39,1.0,10.99,1.0,N,237,264,2,...,1.0,50.00,0.0,0.00,False,False,False,False,2009,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11479975,2,2024-02-01 00:00:17,2024-02-01 00:20:13,1.0,8.93,1.0,N,138,152,2,...,1.0,52.79,0.0,1.75,False,False,False,False,2024,2
11479976,2,2024-02-01 00:01:15,2024-02-01 00:06:30,1.0,1.10,1.0,N,161,234,1,...,1.0,15.48,2.5,0.00,False,False,False,False,2024,2
11479977,2,2024-02-01 00:00:39,2024-02-01 00:12:08,5.0,2.22,1.0,N,186,79,1,...,1.0,22.20,2.5,0.00,False,False,False,False,2024,2
11479978,2,2024-02-01 00:00:17,2024-02-01 00:20:13,1.0,8.93,1.0,N,138,152,2,...,1.0,52.79,0.0,1.75,False,False,False,False,2024,2
